In [1]:
from datetime import datetime, timedelta
import pandas as pd

# Define the start and end dates for the year
start_date = datetime(2026, 1, 1)
end_date = datetime(2027, 1, 31)

# Create an empty list to store the dates
date_list = []
date_list_str = []

# Use a loop to generate dates within the specified range
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date) 
    date_list_str.append(current_date.strftime("%m/%d/%Y")) 
    current_date += timedelta(days=1)

# Create a DataFrame with the dates
date_df = pd.DataFrame(date_list, columns=['DateTime'])
date_df['Date'] = date_list_str

print(start_date)
print(end_date)


2026-01-01 00:00:00
2027-01-31 00:00:00


In [2]:
# Add a column for the day of the week
date_df['Day_of_Week'] = date_df['DateTime'].apply(lambda x: x.strftime('%A'))
date_df['Day_of_Week_Index'] = date_df['DateTime'].dt.weekday

def check_weekday(date):
    day_of_week = date.weekday()
    return day_of_week >= 0 and day_of_week <= 4

date_df['Weekday?'] = date_df['DateTime'].apply(check_weekday)


In [3]:
# create a list of the last days of the month for each month. List is in 2025-01-01 00:00:00 format
last_days = []
for i in range(1, 13):
    last_days.append(date_df[date_df['DateTime'].dt.month == i].iloc[-1]['DateTime'])

for date in last_days:
    # if the date is a weekend, set it to the previous Friday
    orig_date = date
    if date.weekday() >= 5:
        date = date - timedelta(days=(date.weekday() - 4))
    # replace the date in the list
    last_days[last_days.index(orig_date)] = date

print(last_days)

[Timestamp('2027-01-29 00:00:00'), Timestamp('2026-02-27 00:00:00'), Timestamp('2026-03-31 00:00:00'), Timestamp('2026-04-30 00:00:00'), Timestamp('2026-05-29 00:00:00'), Timestamp('2026-06-30 00:00:00'), Timestamp('2026-07-31 00:00:00'), Timestamp('2026-08-31 00:00:00'), Timestamp('2026-09-30 00:00:00'), Timestamp('2026-10-30 00:00:00'), Timestamp('2026-11-30 00:00:00'), Timestamp('2026-12-31 00:00:00')]


In [4]:
# Function to check if a date is in the EOM list
def check_eom(date):
    return date in last_days

date_df['End_of_Month?'] = date_df['DateTime'].apply(check_eom)

In [5]:
# List of 2025 Holidays
holidays = [
    datetime(2026, 1, 1),   # New Year's Day
    datetime(2026, 1, 19),  # Martin Luther King Jr. Day
    datetime(2026, 2, 16),  # Presidents' Day
    datetime(2026, 5, 25),  # Memorial Day
    datetime(2026, 7, 3),   # Independence Day (Observed)
    datetime(2026, 9, 7),   # Labor Day
    datetime(2026, 11, 26), # Thanksgiving Day
    datetime(2026, 12, 25), # Christmas Day
    datetime(2027, 1, 1)    # New Year's Day
    ]

In [6]:
# Function to check if a date is the day after a Holiday
def check_day_after_hol(date):
    return (date + timedelta(days=-1)) in holidays
    

# Apply the check_day_after_hol function to each row in the DataFrame and create a new 'Holiday?' column
date_df['Holiday?'] = date_df['DateTime'].apply(check_day_after_hol)

In [7]:
# Function to determine the File Date based on EOM, Holiday and/or Weekend

def determine_file_date(date):
    # file_date is the date + 1 day
    file_date = date + timedelta(days=1)
    
    flag = False
    while flag == False:
        holiday = check_day_after_hol(file_date)
        eom = check_eom(file_date)
        weekday = check_weekday(file_date)

        if weekday and not holiday and not eom:
            return file_date
        
        file_date += timedelta(days=1)


date_df['File Date'] = date_df['DateTime'].apply(determine_file_date)

In [8]:
# After Cutoff Function to determine the File Date based on EOM, Holiday and/or Weekend 

def determine_file_date(date):
    if date.weekday() == 4:
        file_date = date + timedelta(days=4)
    else:
        file_date = date + timedelta(days=2)
        
    flag = False
    while not flag:
        holiday = check_day_after_hol(file_date)
        eom = check_eom(file_date)
        weekday = check_weekday(file_date)
        # Check the conditions
        if weekday and not holiday and not eom:
            return file_date
        
        # Conditions not met, add 1 day to file_date and continue
        file_date += timedelta(days=1)


date_df['After Cutoff'] = date_df['DateTime'].apply(determine_file_date)

In [9]:
date_df['Formatted FD'] = date_df['File Date'].apply(lambda x: x.strftime('%m/%d/%Y'))
date_df['Formatted Cutoff'] = date_df['After Cutoff'].apply(lambda x: x.strftime('%m/%d/%Y'))
columns_to_drop = ['DateTime', 'File Date', 'After Cutoff','Day_of_Week_Index','Weekday?','End_of_Month?','Holiday?']
date_df.drop(columns=columns_to_drop, inplace=True)

date_df.to_csv("Dates.csv")